In [ ]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import xarray as xr
path = "/Volumes/opto_loc/Data/ACR_39/swi-bl-NNXo.nc"
da = xr.open_dataarray(path)

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
from scipy.stats import shapiro, normaltest
import os
from pathlib import Path

import pingouin as pg

warnings.filterwarnings('ignore')
probe_ord = ['NNXr', 'NNXo']
#hue_ord = [NNXR_GRAY, MAIN_COLOR]
from kdephys.utils.main import td
from cmcrameri import cm as scm

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
import pub_utils as pu
data_agg = acr.utils.import_publication_functions('/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
import data_agg as dag

#--------------------------------- Plotting Setup ---------------------------------#
import pubplots as pp
plt.rcdefaults()
plt.style.use('default')

#plt.rcdefaults()
style_path = '/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr/src/acr/plot_styles/acrvec_labels.mplstyle'
style_path = '/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr/src/acr/plot_styles/acrvec_labels.mplstyle'
#plt.style.use(style_path)

In [ ]:
import kdephys as kde
import yasa
import numpy as np
from yasa import trimbothstd

In [ ]:
from acr.utils import PAPER_FIGURE_ROOT
nbroot = os.path.join(PAPER_FIGURE_ROOT, 'response_to_review', 'spindles')
if not os.path.exists(nbroot):
    os.mkdir(nbroot)

# SOM

In [ ]:
MAIN_EXP = 'swi'
SUBJECT_TYPE = 'som'
subjects, exps = pu.get_subject_list(SUBJECT_TYPE, MAIN_EXP)
MAIN_COLOR = acr.utils.SOM_BLUE
from acr.utils import NNXR_GRAY
MASTER_SCALE = pp.scale(1.2, 1.4)

In [ ]:
lfps = {}
hds = {}
h = {}
for subject, exp in zip(subjects, exps):
    print(subject)
    hds[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True)
    h[subject] = acr.io.load_hypno_full_exp(subject, exp, update=False)

In [ ]:
dfs = []
for probe in ['NNXo', 'NNXr']:
    for cond in ['bl', 'exp']:
        df = pl.read_parquet(f'{acr.utils.rev_data_root}/spindles/{SUBJECT_TYPE}_{cond}_{probe}.parquet')
        dfs.append(df)
df = pl.concat(dfs)

In [ ]:
stimdfs = []
for probe in ['NNXo', 'NNXr']:
    for cond in ['stim']:
        dfstim = pl.read_parquet(f'{acr.utils.rev_data_root}/spindles/{SUBJECT_TYPE}_{cond}_{probe}.parquet')
        stimdfs.append(dfstim)
stim = pl.concat(stimdfs)

In [ ]:
sdfs = []
for subject in subjects:
    sdf = df.filter(pl.col('subject') == subject)
    sdf = acr.hypnogram_utils.label_df_with_states(sdf, h[subject], col='start_time')
    sdf = acr.hypnogram_utils.label_df_with_hypno_conditions(sdf, hds[subject], col='start_time', label_col='hcond')
    sdfs.append(sdf)
df = pl.concat(sdfs)

In [ ]:
stim_dfs = []
for subject in subjects:
    stdf = stim.filter(pl.col('subject') == subject)
    stdf = acr.hypnogram_utils.label_df_with_states(stdf, hds[subject]['stim'], col='start_time')
    stim_only_dur = hds[subject]['stim'].duration.sum()
    stdf = stdf.with_columns(pl.lit(stim_only_dur).alias('stim_only_dur'))
    stdf = stdf.with_columns((pl.col('cond_duration')-pl.col('stim_only_dur')).alias('int_dur'))
    stim_dfs.append(stdf)
stim = pl.concat(stim_dfs)

In [ ]:
sdfs = []
for subject in subjects:
    sdf = df.filter(pl.col('subject') == subject)
    cond_start, cond_end = acr.hypnogram_utils.get_bl_times(hds[subject]['rebound'])
    bl_nrem_dur = h[subject].trim_select(cond_start, cond_end).keep_states(['NREM'])['duration'].sum().total_seconds()
    bl_int_dur = h[subject].trim_select(cond_start, cond_end).keep_states(['Transition-to-REM'])['duration'].sum().total_seconds()
    
    reb_start = hds[subject]['rebound']['start_time'].min()
    reb_end = hds[subject]['rebound']['end_time'].max()
    reb_hypno = h[subject].trim_select(reb_start, reb_end)
    
    reb_nrem_dur = reb_hypno.keep_states(['NREM'])['duration'].sum().total_seconds()
    reb_int_dur = reb_hypno.keep_states(['Transition-to-REM'])['duration'].sum().total_seconds()
    sdf = sdf.with_columns(pl.lit(0).alias('NREM_only_dur'))
    sdf = sdf.with_columns(pl.lit(0).alias('INT_only_dur'))
    sdf = sdf.with_columns(pl.when(pl.col('condition') == 'bl').then(pl.lit(bl_nrem_dur)).otherwise(pl.col('NREM_only_dur')).alias('NREM_only_dur'))
    sdf = sdf.with_columns(pl.when(pl.col('condition') == 'bl').then(pl.lit(bl_int_dur)).otherwise(pl.col('INT_only_dur')).alias('INT_only_dur'))
    sdf = sdf.with_columns(pl.when(pl.col('condition') == 'exp').then(pl.lit(reb_nrem_dur)).otherwise(pl.col('NREM_only_dur')).alias('NREM_only_dur'))
    sdf = sdf.with_columns(pl.when(pl.col('condition') == 'exp').then(pl.lit(reb_int_dur)).otherwise(pl.col('INT_only_dur')).alias('INT_only_dur'))
    sdfs.append(sdf)
df = pl.concat(sdfs)

In [ ]:
df = df.with_columns((pl.col('NREM_only_dur')+pl.col('INT_only_dur')).alias('total_dur'))

In [ ]:
diff_check = df['total_dur'].to_numpy() - df['nrem_dur'].to_numpy()
print(diff_check.max())
if diff_check.max() < 1:
    df = df.drop('nrem_dur')
df = acr.info_pipeline.label_df_sub_infra(df, label_col='depth')

# Spindle Rates, Relative to Full BL

In [ ]:
durdf = df.select(['subject', 'condition', 'NREM_only_dur', 'INT_only_dur', 'total_dur'])
durdf = durdf.group_by(['subject', 'condition']).agg(pl.col(['NREM_only_dur', 'INT_only_dur', 'total_dur']).mean())

In [ ]:
counts = df.group_by(['subject', 'condition', 'probe', 'channel']).agg(pl.count())
counts = counts.join(durdf, on=['subject', 'condition'], how='left')
counts = counts.with_columns((pl.col('count')/pl.col('total_dur')).alias('rate'))
counts = counts.with_columns((pl.col('rate')*60).alias('spm'))
counts = counts.sort(['subject', 'condition', 'probe', 'channel'])

In [ ]:
rel_rates = counts.pivot(
        on='condition',  # or whatever your value column is called
        index=['subject', 'probe'],  # or whatever your subject ID column is called
        values='spm',
        aggregate_function='mean'
    )
rel_rates = rel_rates.with_columns((pl.col('exp') / pl.col('bl')).alias('rel_rate'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min_rel2fbl_AllStates'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxo = rel_rates.prb('NNXo')['rel_rate'].to_numpy()
nnxr = rel_rates.prb('NNXr')['rel_rate'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
is_counts = df.filter(pl.col('state')=='Transition-to-REM').group_by(['subject', 'condition', 'probe', 'channel']).agg(pl.count())
is_counts = is_counts.join(durdf, on=['subject', 'condition'], how='left')
is_counts = is_counts.with_columns((pl.col('count')/pl.col('INT_only_dur')).alias('rate'))
is_counts = is_counts.with_columns((pl.col('rate')*60).alias('spm'))
is_counts = is_counts.sort(['subject', 'condition', 'probe', 'channel'])

In [ ]:
rel_rates = is_counts.pivot(
        on='condition',  # or whatever your value column is called
        index=['subject', 'probe'],  # or whatever your subject ID column is called
        values='spm',
        aggregate_function='mean'
    )
rel_rates = rel_rates.with_columns((pl.col('exp') / pl.col('bl')).alias('rel_rate'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min_rel2fbl_ISonly'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxo = rel_rates.prb('NNXo')['rel_rate'].to_numpy()
nnxr = rel_rates.prb('NNXr')['rel_rate'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
#stat = pg.ttest(nnxr, nnxo, paired=True)
stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "wilcoxon",
    test_statistic=stat["W-val"][0],
    p_value=stat["p-val"][0],
    effect_size_method="RBC",
    effect_size=stat["RBC"][0],
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
nrem_counts = df.filter(pl.col('state')=='NREM').group_by(['subject', 'condition', 'probe', 'channel']).agg(pl.count())
nrem_counts = nrem_counts.join(durdf, on=['subject', 'condition'], how='left')
nrem_counts = nrem_counts.with_columns((pl.col('count')/pl.col('NREM_only_dur')).alias('rate'))
nrem_counts = nrem_counts.with_columns((pl.col('rate')*60).alias('spm'))
nrem_counts = nrem_counts.sort(['subject', 'condition', 'probe', 'channel'])
rel_rates = nrem_counts.pivot(
        on='condition',  # or whatever your value column is called
        index=['subject', 'probe'],  # or whatever your subject ID column is called
        values='spm',
        aggregate_function='mean'
    )
rel_rates = rel_rates.with_columns((pl.col('exp') / pl.col('bl')).alias('rel_rate'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min_rel2fbl_NREMonly'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxo = rel_rates.prb('NNXo')['rel_rate'].to_numpy()
nnxr = rel_rates.prb('NNXr')['rel_rate'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

# Spindle Rates, relative to Circ_bl

In [ ]:
conddf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM').group_by(['subject', 'probe', 'channel', 'hcond']).agg(pl.count())
cdf = conddf.group_by(['subject', 'probe', 'hcond']).agg((pl.col('count').mean()/60).alias('spm')).sort(['subject', 'probe', 'hcond'])

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(16, 6))

# Keep your bar plot for overall means
sns.barplot(data=cdf.prb('NNXr'), x='hcond', y='spm', ax=ax[0], color='gray', alpha=0.5, errorbar=None)

# Add individual subject lines connecting their values across conditions
data = cdf.prb('NNXr')
for subject in data['subject'].unique():  # Replace 'subject' with your actual subject ID column name
    subject_data = data.filter(pl.col('subject') == subject).sort('hcond')
    ax[0].plot(subject_data['hcond'], subject_data['spm'], 
            marker='o', alpha=0.8, color='black', linewidth=2)

# Keep your bar plot for overall means
sns.barplot(data=cdf.prb('NNXo'), x='hcond', y='spm', ax=ax[1], color='lightblue', alpha=0.5, errorbar=None)

# Add individual subject lines connecting their values across conditions
data = cdf.prb('NNXo')
for subject in data['subject'].unique():  # Replace 'subject' with your actual subject ID column name
    subject_data = data.filter(pl.col('subject') == subject).sort('hcond')
    ax[1].plot(subject_data['hcond'], subject_data['spm'], 
            marker='o', alpha=0.8, color=acr.utils.SOM_BLUE, linewidth=2)

In [ ]:
bldf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM')
bldf = bldf.pivot(on='hcond', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='len')
for cond in ['circ_bl', 'early_bl', 'rebound']:
    bldf = bldf.with_columns((pl.col(cond)/60).alias(cond))
bl_means = bldf.group_by(['subject', 'probe']).agg(pl.col(['circ_bl', 'early_bl', 'rebound']).mean()).sort(['subject', 'probe'])

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min__cblVreb__contra-onlyNREM'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')

cbl = bl_means.prb('NNXr')['circ_bl'].to_numpy()
reb = bl_means.prb('NNXr')['rebound'].to_numpy()


cbl = bl_means.prb('NNXr')['circ_bl'].to_numpy()
reb = bl_means.prb('NNXr')['rebound'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(cbl, reb, colors=[NNXR_GRAY, NNXR_GRAY], fsize=pp.scale(MASTER_SCALE), alphas=[0.6, 0.95])
    ax.set_xticklabels(['Circ. BL', 'Rebound'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
nnxr = cbl
nnxo = reb
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "cond": "circ_bl",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "cond": "rebound",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
bldf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM')
bldf = bldf.pivot(on='hcond', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='len')
for cond in ['circ_bl', 'early_bl', 'rebound']:
    bldf = bldf.with_columns((pl.col(cond)/60).alias(cond))
bl_means = bldf.group_by(['subject']).agg(pl.col(['circ_bl', 'early_bl', 'rebound']).mean()).sort(['subject'])

In [ ]:
ebl = bl_means['early_bl'].to_numpy()
cbl = bl_means['circ_bl'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(ebl, cbl)
ax.set_title('Spindle rates, ebl vs cbl, both probes')


In [ ]:
bldf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM')
bldf = bldf.pivot(on='hcond', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='len')
for cond in ['circ_bl', 'early_bl', 'rebound']:
    bldf = bldf.with_columns((pl.col(cond)/60).alias(cond))
bl_means = bldf.group_by(['subject', 'probe']).agg(pl.col(['circ_bl', 'early_bl', 'rebound']).mean()).sort(['subject', 'probe'])
bl_means = bl_means.with_columns((pl.col('rebound')/pl.col('circ_bl')).alias('rebound_rel'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min_rel2circBL_NREMonly'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxr = bl_means.prb('NNXr')['rebound_rel'].to_numpy()
nnxo = bl_means.prb('NNXo')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

## stim

In [ ]:
sns.barplot(stim.filter(pl.col('subject')=='ACR_37').prb('NNXo').group_by('channel').count().sort('channel'), x='channel', y='count')
plt.show()

In [ ]:
conddf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM').group_by(['subject', 'probe', 'channel', 'hcond']).agg(pl.count())
cdf = conddf.group_by(['subject', 'probe', 'channel', 'hcond']).agg((pl.col('count').mean()/60).alias('spm')).sort(['subject', 'probe', 'channel', 'hcond']).prb('NNXo')

In [ ]:
cdf = cdf.filter(pl.col('hcond')=='circ_bl')

In [ ]:
cbl = bl_means.prb('NNXo')['circ_bl'].to_numpy()

In [ ]:
stim_counts = stim.group_by(['subject', 'probe', 'channel', 'state']).agg(pl.count()).sort(['subject', 'probe', 'channel', 'state']).filter(pl.col('state')=='Wake').prb('NNXo')

In [ ]:
new_counts = []
for subject in stim_counts['subject'].unique():
    subdf = stim_counts.filter(pl.col('subject')==subject)
    for channel in np.arange(1, 17):
        if len(subdf.filter(pl.col('channel')==channel)) == 0:
            chan_df = pl.DataFrame({
                'subject': [subject],
                'probe': ['NNXo'],
                'channel': [channel],
                'state': ['Wake'],
                'count': [0]
            })
            # set the count column to uInt32
            chan_df = chan_df.with_columns(pl.col('count').cast(pl.UInt32))
            subdf = subdf.vstack(chan_df)
    new_counts.append(subdf)
stim_counts = pl.concat(new_counts)

In [ ]:
int_counts = stim.group_by(['subject', 'probe', 'channel', 'state']).agg(pl.count()).sort(['subject', 'probe', 'channel', 'state']).filter(pl.col('state')=='no_state').prb('NNXo')

In [ ]:
new_counts = []
for subject in int_counts['subject'].unique():
    subdf = int_counts.filter(pl.col('subject')==subject)
    for channel in np.arange(1, 17):
        if len(subdf.filter(pl.col('channel')==channel)) == 0:
            chan_df = pl.DataFrame({
                'subject': [subject],
                'probe': ['NNXo'],
                'channel': [channel],
                'state': ['no_state'],
                'count': [0]
            })
            # set the count column to uInt32
            chan_df = chan_df.with_columns(pl.col('count').cast(pl.UInt32))
            subdf = subdf.vstack(chan_df)
    new_counts.append(subdf)
int_counts = pl.concat(new_counts)

In [ ]:
stim_durs = stim.group_by('subject').agg(pl.col('stim_only_dur').mean()).sort('subject')

In [ ]:
int_durs = stim.group_by('subject').agg(pl.col('int_dur').mean()).sort('subject')

In [ ]:
stim_counts = stim_counts.join(stim_durs, on='subject')
stim_counts = stim_counts.with_columns((pl.col('count')/(pl.col('stim_only_dur')/60)).alias('rate'))

In [ ]:
int_counts = int_counts.join(int_durs, on='subject')
int_counts = int_counts.with_columns((pl.col('count')/(pl.col('int_dur')/60)).alias('rate'))

In [ ]:
ctx = {}
ctx['ACR_37'] = [9, 10, 11, 12, 13, 14, 15, 16] # clearly something artifactual happening on these channels, likely because of movement-related artifact accounted for them all being on the same row of the omnetics adapter
for subex in ctx.keys():
    # exclude rows from stim_counts where subject==subex and channel in ctx[subex]
    stim_counts = stim_counts.filter(~((pl.col('subject')==subex) & (pl.col('channel').is_in(ctx[subex]))))
    int_counts = int_counts.filter(~((pl.col('subject')==subex) & (pl.col('channel').is_in(ctx[subex]))))
    cdf = cdf.filter(~((pl.col('subject')==subex) & (pl.col('channel').is_in(ctx[subex]))))

In [ ]:
int_counts = int_counts.rename({'rate': 'spm'})
int_counts = int_counts.with_columns(pl.lit('INT').alias('hcond'))
int_counts = int_counts.select(['subject', 'probe', 'channel', 'hcond', 'spm'])

In [ ]:
stim_counts = stim_counts.rename({'rate': 'spm'})
stim_counts = stim_counts.with_columns(pl.lit('stim').alias('hcond'))
stim_counts = stim_counts.select(['subject', 'probe', 'channel', 'hcond', 'spm'])

In [ ]:
fdf = pl.concat([int_counts, stim_counts, cdf])

In [ ]:
for subject in subjects:
    subdf = fdf.filter(pl.col('subject')==subject)
    f, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax = sns.barplot(subdf, x='channel', y='spm', hue='hcond', ax=ax, palette=['gray', 'black', 'blue'], hue_order=['circ_bl', 'INT', 'stim'], order=range(1,17))
    ax.set_title(subject)

    plt.show()

In [ ]:
rmeans = fdf.filter(pl.col('hcond').is_in(['circ_bl', 'stim'])).group_by(['subject', 'hcond']).agg(pl.col('spm').mean()).sort(['subject', 'hcond'])

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spin-per-min__cblVSTIM__OPTRODE-onlyNREM'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')

cbl = rmeans.filter(pl.col('hcond')=='circ_bl')['spm'].to_numpy()
stimdat = rmeans.filter(pl.col('hcond')=='stim')['spm'].to_numpy()

with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(cbl, stimdat, colors=[MAIN_COLOR, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE), alphas=[0.6, 0.95])
    ax.set_xticklabels(['Circ. BL', 'OFF Induction'])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
nnxr = cbl
nnxo = stimdat
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "wilcoxon",
    test_statistic=stat["W-val"][0],
    p_value=stat["p-val"][0],
    effect_size_method="RBC",
    effect_size=stat["RBC"][0],
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "cond": "circ_bl",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "cond": "rebound",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
oodf = pl.read_parquet('/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr_revs/src_dat/oodfs/all_som.parquet')
oodf = oodf.filter(pl.col('off_int')>0.2)
oodf = oodf.cdn('circ_bl')

In [ ]:
cbl_fracs = {}
stim_fracs = {}

In [ ]:
stim_spins = stim.prb('NNXo')
for subex in ctx.keys():
    # exclude rows from stim_counts where subject==subex and channel in ctx[subex]
    stim_spins = stim_spins.filter(~((pl.col('subject')==subex) & (pl.col('channel').is_in(ctx[subex]))))

In [ ]:
df_spins = df.prb('NNXo')
for subex in ctx.keys():
    # exclude rows from stim_counts where subject==subex and channel in ctx[subex]
    df_spins = df_spins.filter(~((pl.col('subject')==subex) & (pl.col('channel').is_in(ctx[subex]))))

In [ ]:
for subject, exp in zip(subjects, exps):
    on_starts = oodf.filter(pl.col('subject')==subject).prb('NNXo')['end_datetime'].to_numpy()
    on_ends = on_starts + np.timedelta64(200, 'ms')

    on_epocs = np.array(list(zip(on_starts, on_ends)))

    spindle_starts = df_spins.filter(pl.col('subject')==subject).prb('NNXo').filter(pl.col('hcond')=='circ_bl')['start_time'].to_numpy()
    # Count spindle starts that fall within any on_epoc
    # Assumption: on_epocs are non-overlapping and sortable by start time.
    # If you suspect overlapping epochs, tell me and I'll drop in a fully general interval-join version.

    on_epocs = np.asarray(on_epocs)
    spindle_starts = np.asarray(spindle_starts)

    # Normalize dtypes (helps avoid datetime64 unit mismatches)
    on_starts = on_epocs[:, 0].astype('datetime64[ns]')
    on_ends = on_epocs[:, 1].astype('datetime64[ns]')
    spindle_starts_ns = spindle_starts.astype('datetime64[ns]')

    # Sort epochs by start time
    order = np.argsort(on_starts)
    on_starts = on_starts[order]
    on_ends = on_ends[order]

    # For each spindle, find the most recent on_start <= spindle_start
    idx = np.searchsorted(on_starts, spindle_starts_ns, side='right') - 1
    valid = idx >= 0

    # Spindle is "in" if it occurs before that epoch's end
    in_on_epoch = np.zeros(spindle_starts_ns.shape[0], dtype=bool)
    in_on_epoch[valid] = spindle_starts_ns[valid] <= on_ends[idx[valid]]

    n_in = int(in_on_epoch.sum())
    n_total = int(spindle_starts_ns.shape[0])
    print(f"{n_in}/{n_total} spindle starts fall inside an on_epoc ({n_in/n_total:.1%})")

    # Optional: how many spindles per on_epoc (epoch index is after sorting)
    per_epoch_counts = np.bincount(idx[in_on_epoch], minlength=len(on_starts))
    print("Spindles per on_epoc (first 10):", per_epoch_counts[:10])

    # Fraction of epochs that contain at least one spindle start
    n_epocs = int(len(on_starts))
    n_epocs_with_spindle = int((per_epoch_counts > 0).sum())
    print(f"{n_epocs_with_spindle}/{n_epocs} on_epocs contain ≥1 spindle ({n_epocs_with_spindle/n_epocs:.1%})")

    # Optional: map each spindle to its on_epoc index (-1 means none)
    spindle_epoch_idx = np.full(n_total, -1, dtype=int)
    spindle_epoch_idx[in_on_epoch] = idx[in_on_epoch]
    frac_epocs = n_epocs_with_spindle/n_epocs
    cbl_fracs[subject] = frac_epocs
    
    # ------------------------------------------------------------------------------------------------------------------------
    
    ss, se, pon, poff, ton, toff = acr.stim.get_all_stim_info(subject, exp, trn_idx=True)
    on_starts = poff
    on_ends = on_starts + np.timedelta64(200, 'ms')
    on_epocs = np.array(list(zip(on_starts, on_ends)))
    spindle_starts = stim_spins.filter(pl.col('subject')==subject).prb('NNXo').filter(pl.col('state')=='Wake')['start_time'].to_numpy()
    # Count spindle starts that fall within any on_epoc
    # Assumption: on_epocs are non-overlapping and sortable by start time.
    # If you suspect overlapping epochs, tell me and I'll drop in a fully general interval-join version.

    on_epocs = np.asarray(on_epocs)
    spindle_starts = np.asarray(spindle_starts)

    # Normalize dtypes (helps avoid datetime64 unit mismatches)
    on_starts = on_epocs[:, 0].astype('datetime64[ns]')
    on_ends = on_epocs[:, 1].astype('datetime64[ns]')
    spindle_starts_ns = spindle_starts.astype('datetime64[ns]')

    # Sort epochs by start time
    order = np.argsort(on_starts)
    on_starts = on_starts[order]
    on_ends = on_ends[order]

    # For each spindle, find the most recent on_start <= spindle_start
    idx = np.searchsorted(on_starts, spindle_starts_ns, side='right') - 1
    valid = idx >= 0

    # Spindle is "in" if it occurs before that epoch's end
    in_on_epoch = np.zeros(spindle_starts_ns.shape[0], dtype=bool)
    in_on_epoch[valid] = spindle_starts_ns[valid] <= on_ends[idx[valid]]

    n_in = int(in_on_epoch.sum())
    n_total = int(spindle_starts_ns.shape[0])
    print(f"{n_in}/{n_total} spindle starts fall inside an on_epoc ({n_in/n_total:.1%})")

    # Optional: how many spindles per on_epoc (epoch index is after sorting)
    per_epoch_counts = np.bincount(idx[in_on_epoch], minlength=len(on_starts))
    print("Spindles per on_epoc (first 10):", per_epoch_counts[:10])

    # Fraction of epochs that contain at least one spindle start
    n_epocs = int(len(on_starts))
    n_epocs_with_spindle = int((per_epoch_counts > 0).sum())
    print(f"{n_epocs_with_spindle}/{n_epocs} on_epocs contain ≥1 spindle ({n_epocs_with_spindle/n_epocs:.1%})")

    # Optional: map each spindle to its on_epoc index (-1 means none)
    spindle_epoch_idx = np.full(n_total, -1, dtype=int)
    spindle_epoch_idx[in_on_epoch] = idx[in_on_epoch]
    frac_epocs = n_epocs_with_spindle/n_epocs
    stim_fracs[subject] = frac_epocs

In [ ]:
cfracs = np.array([cbl_fracs[s] for s in subjects])
sfracs = np.array([stim_fracs[s] for s in subjects])

In [ ]:
fig_name = f'{SUBJECT_TYPE}_on_locked_spindle_rate__cblVSTIM__OPTRODE-only'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')

nnxr = cfracs
nnxo = sfracs

with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[MAIN_COLOR, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE), alphas=[0.6, 0.95])
    ax.set_xticklabels(['Circ. BL', 'OFF Induction'])
    plt.show()
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
nnxr = nnxr
nnxo = nnxo
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
#stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired-Ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "cond": "circ_bl",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "cond": "rebound",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

# Amplitude

## Rel. to Circ. Bl

In [ ]:
ampdf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM')
ampdf = ampdf.pivot(on='hcond', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['circ_bl', 'early_bl', 'rebound']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('rebound')/pl.col('circ_bl')).alias('rebound_rel'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spinAMP_cblVreb__Contra-onlyNREM'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
cbl = ampmeans.prb('NNXr')['circ_bl'].to_numpy()
reb = ampmeans.prb('NNXr')['rebound'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(cbl, reb, colors=[NNXR_GRAY, NNXR_GRAY], fsize=pp.scale(MASTER_SCALE), alphas=[0.6, 0.95])
    ax.set_xticklabels(['Circ. BL', 'Rebound'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
nnxr = cbl
nnxo = reb
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "cond": "circ_bl",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "cond": "rebound",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spinAMP_rel2circBL_NREMonly'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

## Rel. to Full BL

In [ ]:
ampdf = df.filter(pl.col('condition').is_in(['bl', 'exp'])).filter(pl.col('state')=='NREM')
ampdf = ampdf.pivot(on='condition', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['bl', 'exp']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('exp')/pl.col('bl')).alias('rebound_rel'))

In [ ]:
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])

In [ ]:
ampdf = df.filter(pl.col('condition').is_in(['bl', 'exp'])).filter(pl.col('state')=='Transition-to-REM')
ampdf = ampdf.pivot(on='condition', index=['subject', 'probe', 'channel'], values='Amplitude', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['bl', 'exp']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('exp')/pl.col('bl')).alias('rebound_rel'))

In [ ]:
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])

# Duration

In [ ]:
ampdf = df.filter(pl.col('hcond').is_in(['circ_bl', 'early_bl', 'rebound'])).filter(pl.col('state')=='NREM')
ampdf = ampdf.pivot(on='hcond', index=['subject', 'probe', 'channel'], values='Duration', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['circ_bl', 'early_bl', 'rebound']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('rebound')/pl.col('circ_bl')).alias('rebound_rel'))

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spinDUR_cblVreb__Contra-onlyNREM'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
cbl = ampmeans.prb('NNXr')['circ_bl'].to_numpy()
reb = ampmeans.prb('NNXr')['rebound'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(cbl, reb, colors=[NNXR_GRAY, NNXR_GRAY], fsize=pp.scale(MASTER_SCALE), alphas=[0.6, 0.95])
    ax.set_xticklabels(['Circ. BL', 'Rebound'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
nnxr = cbl
nnxo = reb
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "cond": "circ_bl",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "cond": "rebound",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

In [ ]:
fig_name = f'{SUBJECT_TYPE}_spinDUR_rel2circBL_NREMonly'
fig_path = os.path.join(nbroot, f'{fig_name}.svg')
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])
    f.savefig(fig_path, transparent=True, bbox_inches='tight')

In [ ]:
# ===== Adjust here =====
stat_name = fig_name
# =========================

_, p = shapiro(nnxr)
print(f"p-value for NNXr: {p}")

_, p = shapiro(nnxo)
print(f"p-value for NNXo: {p}")

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype="hedges")
stat = pg.ttest(nnxr, nnxo, paired=True)
# stat = pg.wilcoxon(nnxr, nnxo)

acr.stats.write_stats_result(
    stat_name,
    "paired_ttest",
    test_statistic=stat["T"][0],
    p_value=stat["p-val"][0],
    effect_size_method="g",
    effect_size=hg,
    review=True,
)

src1 = pd.DataFrame({
    "spin_rate": nnxr,  # data (e.g. SWA)
    "subject": np.arange(len(nnxr)),  # subject index
    "probe": "NNXr",  # probe
})

src2 = pd.DataFrame({
    "spin_rate": nnxo,  # data (e.g. SWA)
    "subject": np.arange(len(nnxo)),  # subject index
    "probe": "NNXo",  # probe
})
srcdat = pd.concat([src1, src2])
pu.write_source_data(srcdat, stat_name)
stat

## Rel. to Full BL

In [ ]:
ampdf = df.filter(pl.col('condition').is_in(['bl', 'exp'])).filter(pl.col('state')=='NREM')
ampdf = ampdf.pivot(on='condition', index=['subject', 'probe', 'channel'], values='Duration', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['bl', 'exp']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('exp')/pl.col('bl')).alias('rebound_rel'))

In [ ]:
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])

In [ ]:
ampdf = df.filter(pl.col('condition').is_in(['bl', 'exp'])).filter(pl.col('state')=='Transition-to-REM')
ampdf = ampdf.pivot(on='condition', index=['subject', 'probe', 'channel'], values='Duration', aggregate_function='mean')
ampmeans = ampdf.group_by(['subject', 'probe']).agg(pl.col(['bl', 'exp']).mean()).sort(['subject', 'probe'])
ampmeans = ampmeans.with_columns((pl.col('exp')/pl.col('bl')).alias('rebound_rel'))

In [ ]:
nnxo = ampmeans.prb('NNXo')['rebound_rel'].to_numpy()
nnxr = ampmeans.prb('NNXr')['rebound_rel'].to_numpy()
with pp.destination('figma', style=style_path):
    f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=pp.scale(MASTER_SCALE))
    ax.set_xticklabels(['Contra. Control', 'Optrode'])